In [1]:
import json
import os

generated_dir = '../results/2023-01-25_14.40.38.728069'
coco_dir = '../coco'

with open(os.path.join(generated_dir, 'generated.json')) as f:
    generated_pointers = json.load(f) 

In [2]:
generated_pointers[0:20]

[{'image_id': 203564,
  'caption': 'A bicycle replica with a clock as the front wheel.',
  'generated_image': '203564_a-bicycle-replica-with-a-clock-as-the-front-wheel.jpg'},
 {'image_id': 179765,
  'caption': 'A black Honda motorcycle parked in front of a garage.',
  'generated_image': '179765_a-black-honda-motorcycle-parked-in-front-of-a-garage.jpg'},
 {'image_id': 322141,
  'caption': 'A room with blue walls and a white sink and door.',
  'generated_image': '322141_a-room-with-blue-walls-and-a-white-sink-and-door.jpg'},
 {'image_id': 16977,
  'caption': 'A car that seems to be parked illegally behind a legally parked car',
  'generated_image': '16977_a-car-that-seems-to-be-parked-illegally-behind-a-legally-parked-car.jpg'},
 {'image_id': 106140,
  'caption': 'A large passenger airplane flying through the air.',
  'generated_image': '106140_a-large-passenger-airplane-flying-through-the-air.jpg'},
 {'image_id': 106140,
  'caption': 'There is a GOL plane taking off in a partly cloudy s

In [3]:
"""
CLIP-score doesn't actually need the real image; I am just testing that we are able to locate it.
We will need this for FID-score.
"""

def get_real_image_path(coco_dir, image_id):
    image_id_str = str(image_id).zfill(12) # pad to 12 characters
    image_base_fname = f"COCO_val2014_{image_id_str}.jpg"
    image_path = os.path.join(coco_dir, 'val2014', image_base_fname)
    return image_path

real_image_path = get_real_image_path(coco_dir, generated_pointers[0]['image_id'])

real_image_path

'../coco/val2014/COCO_val2014_000000203564.jpg'

In [ ]:
# CLIP score demo

import torch
from torchvision.io import read_image
import torchvision.transforms as T
from torchmetrics.multimodal import CLIPScore


metric = CLIPScore(model_name_or_path="openai/clip-vit-base-patch16") # TODO: which CLIP model do others use?

score_noise = metric(torch.randint(255, (3, 224, 224)), "a photo of a cat")
print(f"score_noise: {score_noise}")

sum_clip_score = 0

for i in range(len(generated_pointers)):
    generated_image_path = os.path.join(generated_dir, generated_pointers[i]['generated_image'])
    generated_image = read_image(generated_image_path)

    # img = T.ToPILImage()(generated_image)
    # img.show()

    score_actual = metric(generated_image, generated_pointers[i]['caption'])
    print(f"score {i}: {score_actual}")

mean_clip_score = sum_clip_score / len(generated_pointers)

/home/forrest/anaconda3/envs/nov2022_diffusers/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


score_noise: 24.652740478515625
score 0: 29.513614654541016
score 1: 28.45810890197754
score 2: 34.60138702392578
score 3: 24.628847122192383
score 4: 30.42526626586914
score 5: 24.69168472290039
score 6: 37.36733627319336
score 7: 34.12033462524414
score 8: 33.41608810424805
score 9: 27.333444595336914
score 10: 30.239782333374023
score 11: 29.865205764770508
score 12: 26.857927322387695
score 13: 34.22323989868164
score 14: 30.690690994262695
score 15: 32.1330680847168
score 16: 32.03816223144531
score 17: 29.20905303955078
score 18: 31.937095642089844
score 19: 32.5599365234375
score 20: 30.953853607177734
score 21: 35.94513702392578
score 22: 35.440895080566406
score 23: 28.53565788269043
score 24: 29.20604705810547
score 25: 31.601783752441406
score 26: 29.59362030029297
score 27: 28.22239112854004
score 28: 36.76041793823242
score 29: 26.873912811279297
score 30: 31.808015823364258
score 31: 35.722328186035156
score 32: 36.527652740478516
score 33: 33.458377838134766
score 34: 33

score 281: 35.49998092651367
score 282: 28.447124481201172
score 283: 29.322237014770508
score 284: 30.824005126953125
score 285: 24.145153045654297
score 286: 30.974260330200195
score 287: 33.86363983154297
score 288: 30.02682876586914
score 289: 32.963253021240234
score 290: 32.76393127441406
score 291: 28.708181381225586
score 292: 28.515247344970703
score 293: 27.896209716796875
score 294: 36.448158264160156
score 295: 30.519939422607422
score 296: 31.106281280517578
score 297: 32.54728698730469
score 298: 28.75385284423828
score 299: 31.313884735107422
score 300: 30.413034439086914
score 301: 30.585237503051758
score 302: 32.411338806152344
score 303: 32.76788330078125
score 304: 29.890342712402344
score 305: 29.989614486694336
score 306: 30.040613174438477
score 307: 31.372785568237305
score 308: 33.25032424926758
score 309: 34.719383239746094
score 310: 31.764915466308594
score 311: 34.07754898071289
score 312: 28.066503524780273
score 313: 30.41609764099121
score 314: 30.691698

score 560: 31.79412269592285
score 561: 27.63795280456543
score 562: 28.78141975402832
score 563: 34.31969451904297
score 564: 34.51414489746094
score 565: 33.19735336303711
score 566: 38.07810974121094
score 567: 33.84248352050781
score 568: 33.686466217041016
score 569: 30.367942810058594
score 570: 33.825199127197266
score 571: 32.37663650512695
score 572: 27.416271209716797
score 573: 26.50066566467285
score 574: 32.820064544677734
score 575: 29.054515838623047
score 576: 32.44753646850586
score 577: 32.272518157958984
score 578: 31.199565887451172
score 579: 32.9683723449707
score 580: 31.93775749206543
score 581: 32.347206115722656
score 582: 29.79184913635254
score 583: 33.41780090332031
score 584: 28.552160263061523
score 585: 38.42380142211914
score 586: 30.033672332763672
score 587: 26.030614852905273
score 588: 34.5933723449707
score 589: 30.28426742553711
score 590: 31.702253341674805
score 591: 36.43449783325195
score 592: 29.70668601989746
score 593: 29.118717193603516
sc

score 839: 26.56072998046875
score 840: 38.14090347290039
score 841: 33.09804916381836
score 842: 30.61138916015625
score 843: 34.01885223388672
score 844: 35.45744705200195
score 845: 32.86632537841797
score 846: 36.21926498413086
score 847: 31.00511932373047
score 848: 31.166114807128906
score 849: 30.8206787109375
score 850: 27.71399688720703
score 851: 31.55752944946289
score 852: 31.329280853271484
score 853: 32.24883270263672
score 854: 34.00130081176758
score 855: 31.04085922241211
score 856: 34.43574523925781
score 857: 33.982452392578125
score 858: 31.287025451660156
score 859: 32.38129425048828
score 860: 32.275474548339844
score 861: 27.47989845275879
score 862: 30.394208908081055
score 863: 25.73247528076172
score 864: 32.404361724853516
score 865: 30.570390701293945
score 866: 32.01237487792969
score 867: 31.461626052856445
score 868: 30.97620964050293
score 869: 35.65865707397461
score 870: 31.83220672607422
score 871: 32.90739822387695
score 872: 31.034759521484375
score

...

score 11324: 37.4403076171875
score 11325: 33.687477111816406
score 11326: 30.487092971801758
score 11327: 32.129600524902344
score 11328: 32.20751190185547
score 11329: 31.72991180419922
score 11330: 31.399456024169922
score 11331: 30.87643051147461
score 11332: 34.65167999267578
score 11333: 27.738548278808594
score 11334: 32.20974349975586
score 11335: 27.3853816986084
score 11336: 33.16952133178711
score 11337: 28.33946418762207
score 11338: 38.89253234863281
score 11339: 30.178749084472656
score 11340: 32.875125885009766
score 11341: 29.513439178466797
score 11342: 35.184730529785156
score 11343: 35.19686508178711
score 11344: 26.025808334350586
score 11345: 37.026275634765625
score 11346: 35.91144561767578
score 11347: 27.830276489257812
score 11348: 33.381935119628906
score 11349: 32.12702560424805
score 11350: 31.503238677978516
score 11351: 31.587249755859375
score 11352: 35.94648361206055
score 11353: 24.71836280822754
score 11354: 28.846332550048828
score 11355: 33.945663452

score 11585: 34.44562911987305
score 11586: 35.12327194213867
score 11587: 34.18296813964844
score 11588: 29.436189651489258
score 11589: 26.06846046447754
score 11590: 27.51380729675293
score 11591: 26.869613647460938
score 11592: 26.9436092376709
score 11593: 33.64316177368164
score 11594: 34.7704963684082
score 11595: 34.13656234741211
score 11596: 30.210546493530273
score 11597: 30.825212478637695
score 11598: 28.852943420410156
score 11599: 32.1502799987793
score 11600: 32.150550842285156
score 11601: 28.933685302734375
score 11602: 28.23599624633789
score 11603: 34.17054748535156
score 11604: 29.500747680664062
score 11605: 39.0683479309082
score 11606: 30.593978881835938
score 11607: 29.523849487304688
score 11608: 30.037521362304688
score 11609: 36.40725326538086
score 11610: 34.617427825927734
score 11611: 31.4518985748291
score 11612: 30.610679626464844
score 11613: 27.293649673461914
score 11614: 35.23616027832031
score 11615: 32.17798614501953
score 11616: 26.29376029968261

score 11846: 31.26285171508789
score 11847: 30.809635162353516
score 11848: 33.096763610839844
score 11849: 29.316579818725586
score 11850: 29.378196716308594
score 11851: 31.804157257080078
score 11852: 28.19743537902832
score 11853: 29.720861434936523
score 11854: 28.996610641479492
score 11855: 31.222183227539062
score 11856: 28.10240364074707
score 11857: 32.3244514465332
score 11858: 34.099395751953125
score 11859: 32.928951263427734
score 11860: 35.228519439697266
score 11861: 37.19335174560547
score 11862: 29.011127471923828
score 11863: 30.118619918823242
score 11864: 32.385719299316406
score 11865: 31.24624252319336
score 11866: 29.097179412841797
score 11867: 32.63727951049805
score 11868: 26.20160675048828
score 11869: 29.15144920349121
score 11870: 32.13672637939453
score 11871: 32.349945068359375
score 11872: 30.72057342529297
score 11873: 30.48661994934082
score 11874: 31.38176155090332
score 11875: 34.232479095458984
score 11876: 31.248929977416992
score 11877: 29.588464

score 12107: 27.473255157470703
score 12108: 31.71239471435547
score 12109: 31.834741592407227
score 12110: 34.064083099365234
score 12111: 32.9739875793457
score 12112: 28.86272621154785
score 12113: 31.792808532714844
score 12114: 33.5575065612793
score 12115: 29.173542022705078
score 12116: 31.688426971435547
score 12117: 29.846229553222656
score 12118: 30.685705184936523
score 12119: 33.76894760131836
score 12120: 31.87808609008789
score 12121: 28.181575775146484
score 12122: 30.648841857910156
score 12123: 38.590545654296875
score 12124: 30.88492202758789
score 12125: 40.25961685180664
score 12126: 35.60698318481445
score 12127: 25.70279884338379
score 12128: 31.656789779663086
score 12129: 32.33759689331055
score 12130: 37.4763298034668
score 12131: 31.701416015625
score 12132: 33.40874481201172
score 12133: 31.868911743164062
score 12134: 38.65755081176758
score 12135: 30.52572250366211
score 12136: 33.51091766357422
score 12137: 30.51818084716797
score 12138: 34.013099670410156

score 12368: 28.716960906982422
score 12369: 31.529033660888672
score 12370: 35.91456604003906
score 12371: 32.9483642578125
score 12372: 35.560142517089844
score 12373: 31.103273391723633
score 12374: 29.98436164855957
score 12375: 37.385986328125
score 12376: 31.28451919555664
score 12377: 31.698040008544922
score 12378: 35.06364059448242
score 12379: 32.13831329345703
score 12380: 29.958480834960938
score 12381: 32.98303985595703
score 12382: 32.073463439941406
score 12383: 28.567785263061523
score 12384: 30.33353614807129
score 12385: 34.28311538696289
score 12386: 30.79181671142578
score 12387: 34.335426330566406
score 12388: 37.170894622802734
score 12389: 30.55167579650879
score 12390: 33.97383117675781
score 12391: 25.83818817138672
score 12392: 29.55951690673828
score 12393: 26.232524871826172
score 12394: 29.926305770874023
score 12395: 35.29788589477539
score 12396: 32.36958694458008
score 12397: 31.06618309020996
score 12398: 34.98624038696289
score 12399: 33.88988494873047

In [ ]:
# prep for FID score demo

generated_images = []
real_images = []

for i in range(len(generated_pointers)):
    image_path = get_real_image_path(coco_dir, generated_pointers[i]['image_id'])
    # print(image_path)
    real_images.append(read_image(image_path))

for i in range(len(generated_pointers)):
    image_path = os.path.join(generated_dir, generated_pointers[i]['generated_image'])
    generated_images.append(read_image(image_path))


    

In [ ]:
img = real_images[0]
img.shape
img2 = img.unsqueeze(0)
img2.shape

In [ ]:
# FID score demo

from torchmetrics.image.fid import FrechetInceptionDistance

fid = FrechetInceptionDistance(feature=2048)

"""
We are running the following with batch size 1. It would likely be faster to use a larger batch size.
But, the real images have various resolutions and aspect ratios.
It's convenient to let torchmetrics take care of resizing the images.
"""

i = 0
for generated, real in zip(generated_images, real_images):
    fid.update(generated.unsqueeze(0), real=False)
    fid.update(real.unsqueeze(0), real=True)
    
    if i%100 == 0:
        print(f"fid score iteration {i}")
    
    i += 1

fid_score = fid.compute()
print(f"fid score: {fid_score}")